### Problem 1

In [67]:
import torch
import torch.nn as nn


# fix seed so that random initialization always performs the same 
torch.manual_seed(13)


# create the model N as described in the question
N = nn.Sequential(nn.Linear(10, 10, bias=False),
                  nn.ReLU(),
                  nn.Linear(10, 10, bias=False),
                  nn.ReLU(),
                  nn.Linear(10, 3, bias=False))

# random input
x = torch.rand((1,10)) # the first dimension is the batch size; the following dimensions the actual dimension of the data
x.requires_grad_() # this is required so we can compute the gradient w.r.t x

t = 0 # target class

epsReal = 0.5  #depending on your data this might be large or small
eps = epsReal - 1e-7 # small constant to offset floating-point erros

# The network N classfies x as belonging to class 2
original_class = N(x).argmax(dim=1).item()  # TO LEARN: make sure you understand this expression
print("Original Class: ", original_class)
assert(original_class == 2)

# compute gradient
# note that CrossEntropyLoss() combines the cross-entropy loss and an implicit softmax function
L = nn.CrossEntropyLoss()
loss = L(N(x), torch.tensor([t], dtype=torch.long)) # TO LEARN: make sure you understand this line
loss.backward()

# your code here
# adv_x should be computed from x according to the fgsm-style perturbation such that the new class of xBar is the target class t above
# hint: you can compute the gradient of the loss w.r.t to x as x.grad
adv_x = x - eps * x.grad.sign()

new_class = N(adv_x).argmax(dim=1).item()
print("New Class: ", new_class)
assert(new_class == t)
# it is not enough that adv_x is classified as t. We also need to make sure it is 'close' to the original x. 
print(torch.norm((x-adv_x),  p=float('inf')).data)
assert( torch.norm((x-adv_x), p=float('inf')) <= epsReal)

Original Class:  2
New Class:  0
tensor(0.5000)


Iteratively update adv_x using a smaller step size

In [68]:
import torch
import torch.nn as nn


# fix seed so that random initialization always performs the same
torch.manual_seed(13)


# create the model N as described in the question
N = nn.Sequential(nn.Linear(10, 10, bias=False),
                  nn.ReLU(),
                  nn.Linear(10, 10, bias=False),
                  nn.ReLU(),
                  nn.Linear(10, 3, bias=False))

# random input
x = torch.rand((1,10)) # the first dimension is the batch size; the following dimensions the actual dimension of the data
x.requires_grad_() # this is required so we can compute the gradient w.r.t x

t = 1 # target class


alpha = 0.01 # step size
epsReal = 0.9  #depending on your data this might be large or small
eps = epsReal - 1e-7 # small constant to offset floating-point erros

# The network N classfies x as belonging to class 2
original_class = N(x).argmax(dim=1).item()  # TO LEARN: make sure you understand this expression
print("Original Class: ", original_class)
assert(original_class == 2)

new_class = original_class
adv_x = x.clone().detach()
adv_x.requires_grad_()
while new_class!=t:
    adv_x.grad=None
    # compute gradient
    # note that CrossEntropyLoss() combines the cross-entropy loss and an implicit softmax function
    L = nn.CrossEntropyLoss()
    loss = L(N(adv_x), torch.tensor([t], dtype=torch.long)) # TO LEARN: make sure you understand this line
    loss.backward()

    # your code here
    # adv_x should be computed from x according to the fgsm-style perturbation such that the new class of xBar is the target class t above
    # hint: you can compute the gradient of the loss w.r.t to x as x.grad
    adv_x = adv_x - alpha * adv_x.grad
    adv_x = adv_x.clamp(x - eps, x + eps).detach().requires_grad_()
    new_class = N(adv_x).argmax(dim=1).item()

print("New Class: ", new_class)
assert(new_class == t)
# it is not enough that adv_x is classified as t. We also need to make sure it is 'close' to the original x.
print(torch.norm((x-adv_x),  p=float('inf')).data)
assert( torch.norm((x-adv_x), p=float('inf')) <= epsReal)

Original Class:  2
New Class:  1
tensor(0.8207)


### Problem 2

#### Set up for dataset and model

Package installation, loading, and dataloaders. There's also a resnet18 model defined.

In [69]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
# train_dataset = datasets.CIFAR10('cifar10_data/', train=True, download=True, transform=transforms.Compose(
#     [transforms.ToTensor()]
# ))
test_dataset = datasets.CIFAR10('cifar10_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



Files already downloaded and verified


In [70]:

def tp_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return .5 * (x + delta) * (1 - ind1) * (1 - ind2) + x * ind2

def tp_smoothed_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return (x + delta) ** 2 / (4 * delta) * (1 - ind1) * (1 - ind2) + x * ind2

class Normalize(nn.Module):
    def __init__(self, mu, std):
        super(Normalize, self).__init__()
        self.mu, self.std = mu, std

    def forward(self, x):
        return (x - self.mu) / self.std

class IdentityLayer(nn.Module):
    def forward(self, inputs):
        return inputs
    
class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, bn, learnable_bn, stride=1, activation='relu'):
        super(PreActBlock, self).__init__()
        self.collect_preact = True
        self.activation = activation
        self.avg_preacts = []
        self.bn1 = nn.BatchNorm2d(in_planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=not learnable_bn)
        self.bn2 = nn.BatchNorm2d(planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=not learnable_bn)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=not learnable_bn)
            )

    def act_function(self, preact):
        if self.activation == 'relu':
            act = F.relu(preact)
        elif self.activation[:6] == '3prelu':
            act = tp_relu(preact, delta=float(self.activation.split('relu')[1]))
        elif self.activation[:8] == '3psmooth':
            act = tp_smoothed_relu(preact, delta=float(self.activation.split('smooth')[1]))
        else:
            assert self.activation[:8] == 'softplus'
            beta = int(self.activation.split('softplus')[1])
            act = F.softplus(preact, beta=beta)
        return act

    def forward(self, x):
        out = self.act_function(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x  # Important: using out instead of x
        out = self.conv1(out)
        out = self.conv2(self.act_function(self.bn2(out)))
        out += shortcut
        return out

class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, n_cls, cuda=True, half_prec=False,
        activation='relu', fts_before_bn=False, normal='none'):
        super(PreActResNet, self).__init__()
        self.bn = True
        self.learnable_bn = True  # doesn't matter if self.bn=False
        self.in_planes = 64
        self.avg_preact = None
        self.activation = activation
        self.fts_before_bn = fts_before_bn
        if normal == 'cifar10':
            self.mu = torch.tensor((0.4914, 0.4822, 0.4465)).view(1, 3, 1, 1)
            self.std = torch.tensor((0.2471, 0.2435, 0.2616)).view(1, 3, 1, 1)
        else:
            self.mu = torch.tensor((0.0, 0.0, 0.0)).view(1, 3, 1, 1)
            self.std = torch.tensor((1.0, 1.0, 1.0)).view(1, 3, 1, 1)
            print('no input normalization')
        if cuda:
            self.mu = self.mu.cuda()
            self.std = self.std.cuda()
        if half_prec:
            self.mu = self.mu.half()
            self.std = self.std.half()

        self.normalize = Normalize(self.mu, self.std)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=not self.learnable_bn)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.bn = nn.BatchNorm2d(512 * block.expansion)
        self.linear = nn.Linear(512*block.expansion, n_cls)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, self.bn, self.learnable_bn, stride, self.activation))
            # layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        for layer in [*self.layer1, *self.layer2, *self.layer3, *self.layer4]:
            layer.avg_preacts = []

        out = self.normalize(x)
        out = self.conv1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        if return_features and self.fts_before_bn:
            return out.view(out.size(0), -1)
        out = F.relu(self.bn(out))
        if return_features:
            return out.view(out.size(0), -1)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


def PreActResNet18(n_cls, cuda=True, half_prec=False, activation='relu', fts_before_bn=False,
    normal='none'):
    #print('initializing PA RN-18 with act {}, normal {}'.format())
    return PreActResNet(PreActBlock, [2, 2, 2, 2], n_cls=n_cls, cuda=cuda, half_prec=half_prec,
        activation=activation, fts_before_bn=fts_before_bn, normal=normal)


# intialize the model
model = PreActResNet18(10, cuda=True, activation='softplus1').to(device)
model.eval()

no input normalization


PreActResNet(
  (normalize): Normalize()
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
  )
  (layer2): Sequential(
    (0): PreActBloc

#### Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [71]:
def pgd_linf_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach().to(device)
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        # TODO: compute the adv_x
        # find delta, clamp with eps
        with torch.no_grad():
            adv_x = adv_x + eps_step * adv_x.grad.sign()
            delta = adv_x - x
            delta = torch.clamp(delta, -eps, eps)
            adv_x = torch.clamp(x + delta, min=0, max=1).detach()
    return adv_x

In [72]:
def pgd_l2_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        batch_size = x.size()[0]
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        grad = adv_x.grad.data
        # TODO: compute the adv_x
        # find delta, clamp with eps, project delta to the l2 ball
        # HINT: https://github.com/Harry24k/adversarial-attacks-pytorch/blob/master/torchattacks/attacks/pgdl2.py
        with torch.no_grad():
            grad_norms = (
                torch.norm(grad.view(batch_size, -1), p=2, dim=1) + 1e-10
            )
            adv_x = adv_x + eps_step * grad / grad_norms.view(-1, 1, 1, 1)
            delta = adv_x - x
            delta_norms = torch.norm(delta.view(batch_size, -1), p=2, dim=1)
            factor = eps / delta_norms
            factor = torch.min(factor, torch.ones_like(delta_norms))
            delta = delta * factor.view(-1, 1, 1, 1)
            adv_x = torch.clamp(x + delta, min=0, max=1).detach()

    return adv_x

#### Evaluate Single and Multi-Norm Robust Accuracy

In this section, we evaluate the model on the Linf and L2 attacks as well as union accuracy.

In [73]:
def test_model_on_single_attack(model, attack='pgd_linf', eps=0.1):
    model.eval()
    tot_test, tot_acc, orig_acc = 0.0, 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        if attack == 'pgd_linf':
            # TODO: get x_adv untargeted pgd linf with eps, and eps_step=eps/4
            x_adv = pgd_linf_untargeted(model, x_batch, y_batch, k=10, eps=eps, eps_step=eps/4)
        elif attack == 'pgd_l2':
            # TODO: get x_adv untargeted pgd l2 with eps, and eps_step=eps/4
            x_adv = pgd_l2_untargeted(model, x_batch, y_batch, k=10, eps=eps, eps_step=eps/4)
        else:
            pass

        # get the testing accuracy and update tot_test and tot_acc
        with torch.no_grad():
            out = model(x_adv)
            tot_acc += (out.argmax(dim=1) == y_batch).sum().item()
            tot_test += x_batch.size(0)
            orig_acc += (model(x_batch).argmax(dim=1) == y_batch).sum().item()

    print('Standard accuracy %.5lf' % (orig_acc/tot_test), 'Robust accuracy %.5lf' % (tot_acc/tot_test), f'on {attack} attack with eps = {eps}')
    

## Single-Norm Robust Accuracy

In [74]:
# Evaluate on Linf attack with different models with eps = 8/255
model.load_state_dict(torch.load('models/pretr_Linf.pth'))
# Evaluate on Linf attack with model 1 with eps = 8/255
test_model_on_single_attack(model, attack='pgd_linf', eps=8/255)
model.load_state_dict(torch.load('models/pretr_L2.pth'))
# Evaluate on Linf attack with model 2 with eps = 8/255
test_model_on_single_attack(model, attack='pgd_linf', eps=8/255)
model.load_state_dict(torch.load('models/pretr_RAMP.pth'))
# Evaluate on Linf attack with model 3 with eps = 8/255
test_model_on_single_attack(model, attack='pgd_linf', eps=8/255)

Evaluating: 100%|██████████| 157/157 [01:24<00:00,  1.85it/s]


Standard accuracy 0.82800 Robust accuracy 0.51200 on pgd_linf attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [01:25<00:00,  1.84it/s]


Standard accuracy 0.88750 Robust accuracy 0.30860 on pgd_linf attack with eps = 0.03137254901960784


Evaluating: 100%|██████████| 157/157 [01:26<00:00,  1.82it/s]

Standard accuracy 0.81190 Robust accuracy 0.49740 on pgd_linf attack with eps = 0.03137254901960784


In [75]:
# Evaluate on L2 attack with different models with eps = 0.75
model.load_state_dict(torch.load('models/pretr_Linf.pth'))
# Evaluate on L2 attack with model 1 with eps = 0.75
test_model_on_single_attack(model, attack="pgd_l2", eps=0.75)
model.load_state_dict(torch.load('models/pretr_L2.pth'))
# Evaluate on L2 attack with model 2 with eps = 0.75
test_model_on_single_attack(model, attack="pgd_l2", eps=0.75)
model.load_state_dict(torch.load('models/pretr_RAMP.pth'))
# Evaluate on L2 attack with model 3 with eps = 0.75
test_model_on_single_attack(model, attack="pgd_l2", eps=0.75)

Evaluating: 100%|██████████| 157/157 [01:26<00:00,  1.81it/s]


Standard accuracy 0.82800 Robust accuracy 0.48820 on pgd_l2 attack with eps = 0.75


Evaluating: 100%|██████████| 157/157 [01:26<00:00,  1.82it/s]


Standard accuracy 0.88750 Robust accuracy 0.54390 on pgd_l2 attack with eps = 0.75


Evaluating: 100%|██████████| 157/157 [01:26<00:00,  1.82it/s]

Standard accuracy 0.81190 Robust accuracy 0.59660 on pgd_l2 attack with eps = 0.75


## Multi-Norm Robust Accuracy

In [76]:
def test_model_on_multi_attacks(model, eps_linf=8./255., eps_l2=0.75):
    model.eval()
    tot_test, tot_acc, orig_acc = 0.0, 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        # TODO: get x_adv_linf and x_adv_l2 untargeted pgd linf and l2 with eps, and eps_step=eps/4
        x_adv_linf = pgd_linf_untargeted(model, x_batch, y_batch, k=10, eps=eps_linf, eps_step=eps_linf/4)
        x_adv_l2 = pgd_l2_untargeted(model, x_batch, y_batch, k=10, eps=eps_l2, eps_step=eps_l2/4)
        ## calculate union accuracy: correct only if both attacks are correct
        with torch.no_grad():
            out = model(x_adv_linf)
            pred_linf = torch.max(out, dim=1)[1]
            out = model(x_adv_l2)
            pred_l2 = torch.max(out, dim=1)[1]

            orig_acc += (model(x_batch).argmax(dim=1) == y_batch).sum().item()
            # TODO: get the testing accuracy with multi-norm robustness and update tot_test and tot_acc
            tot_acc += ((pred_linf == y_batch) & (pred_l2 == y_batch)).sum().item()
            tot_test += x_batch.size(0)

    print(
        "Standard accuracy %.5lf" % (orig_acc / tot_test),
        "Robust accuracy %.5lf" % (tot_acc / tot_test),
        f"on multi attacks",
    )

In [77]:
# Evaluate on multi-norm attacks with different models with eps_linf = 8./255, eps_l2 = 0.75
model.load_state_dict(torch.load('models/pretr_Linf.pth'))
# Evaluate on multi attacks with model 1
test_model_on_multi_attacks(model, eps_linf=8./255., eps_l2=0.75)
model.load_state_dict(torch.load('models/pretr_L2.pth'))
# Evaluate on multi attacks with model 2
test_model_on_multi_attacks(model, eps_linf=8./255., eps_l2=0.75)
model.load_state_dict(torch.load('models/pretr_RAMP.pth'))
# Evaluate on multi attacks with model 3
test_model_on_multi_attacks(model, eps_linf=8./255., eps_l2=0.75)

Evaluating: 100%|██████████| 157/157 [02:48<00:00,  1.07s/it]


Standard accuracy 0.82800 Robust accuracy 0.47040 on multi attacks


Evaluating: 100%|██████████| 157/157 [02:48<00:00,  1.07s/it]


Standard accuracy 0.88750 Robust accuracy 0.30860 on multi attacks


Evaluating: 100%|██████████| 157/157 [02:48<00:00,  1.07s/it]

Standard accuracy 0.81190 Robust accuracy 0.49730 on multi attacks
